<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=1c4da771f26f2439276f4a2ae0fc0dc6040adb31b6ad6baf6aa61c0ba6bf256f
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [2]:
import librosa
import numpy as np
import pandas as pd
from time import time
from glob import glob
import tensorflow as tf
from tensorflow import keras 
import python_speech_features
import matplotlib.pyplot as plt
from scipy.signal.windows import hamming
from sklearn.metrics import classification_report,  plot_confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
np.random.seed(1)
tf.random.set_seed(1)

In [4]:
# Upload audio files from github
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 1580, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 1580 (delta 60), reused 96 (delta 39), pack-reused 1444
Receiving objects: 100% (1580/1580), 237.98 MiB | 22.13 MiB/s, done.
Resolving deltas: 100% (208/208), done.
Checking out files: 100% (1145/1145), done.


In [6]:
# Name of vegetables and fruits 52 types of 56 classes
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย','ผักขึ้นช่าย','ผักชะอม','ผักชี','ผักชีฝรั่ง','ผักตำลึง',
               'มะปราง','มะพลับ','มะละกอ','มะกรูด','มะเขือพวง','มะเขือเทศ','มะระ','มะรุม','มะตูม','มันแกว1','มันแกว2','มันเทศ','มันฝรั่ง','มันสำปะหลัง',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด','พุทรา','เพกา',
               'ฝรั่ง',
               'ฟัก','ฟักทอง',
               'สับปะรด','สะเดา','สะระแหน่',
               'หัวหอม1','หัวหอม2']

# Audio data source
source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [7]:
def feature_mfcc(y,sr):
    return (python_speech_features.mfcc(signal=y, 
                                    samplerate=sr, 
                                    winlen= 512/sr, 
                                    winstep= 160/sr,
                                    numcep= 13,
                                    nfilt= 40, 
                                    nfft= 512,
                                    lowfreq= 0,
                                    highfreq= None,
                                    preemph= 0.97, 
                                    ceplifter= 0,
                                    winfunc= hamming))

In [8]:
for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=5,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(feature_mfcc(y,sr))
        source['label'].append(fruit_veget.index(types))

In [9]:
# tranform dictionary to dataframe 
df = pd.DataFrame.from_dict(source)
df

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.00021362305, -0.00024414062, -0.0002593994...",22050,"[[-14.389836672385517, -10.023800010987056, 0....",0,กระชาย
1,"[-1.5258789e-05, 1.5258789e-05, -1.5258789e-05...",22050,"[[-13.834222688815137, -11.278032760622157, -1...",0,กระชาย
2,"[-9.1552734e-05, -0.00012207031, -0.0001373291...",22050,"[[-13.95247445826829, -10.979968172429752, 1.8...",0,กระชาย
3,"[0.0, 0.0, 0.0, 3.0517578e-05, -3.0517578e-05,...",22050,"[[-14.175309888089249, -10.723428500252963, 1....",0,กระชาย
4,"[-7.6293945e-05, -0.00015258789, -0.0001068115...",22050,"[[-13.963624272836848, -10.834306371234895, 4....",0,กระชาย
...,...,...,...,...,...
1115,"[-0.00018310547, -0.0002593994, -0.00021362305...",22050,"[[-14.051788496188221, -10.373070308117178, 4....",55,หัวหอม2
1116,"[-0.00024414062, -0.0002746582, -0.00019836426...",22050,"[[-14.30657814127106, -8.777075651980645, 2.11...",55,หัวหอม2
1117,"[0.0, 0.0, -3.0517578e-05, 3.0517578e-05, -3.0...",22050,"[[-13.937844881809626, -12.453958235841547, -0...",55,หัวหอม2
1118,"[-0.00024414062, -0.0002593994, -0.00022888184...",22050,"[[-14.003234060711906, -11.759421272003904, -1...",55,หัวหอม2


In [10]:
df.to_csv('audio_data.csv')